In [2]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from common import Data

In [5]:
stats = Data.get_nwsl_player_stats(include_playoffs=False)
stats

,season,date,home,away,player,team,number,position,is_starter,accurate_keeper_throws,...,att_obp_goal,att_pen_miss,assist_post,att_lg_left,back_pass,corner_kicks,penalty_saved,has_full_stats,goal_assists,is_playoffs
0,2016,2016-04-16,HOU,CHI,Alyssa Naeher,CHI,1,Goalkeeper,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False
1,2016,2016-04-16,HOU,CHI,Casey Krueger,CHI,6,Defender,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False
2,2016,2016-04-16,HOU,CHI,Julie Ertz,CHI,8,Defender,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False
3,2016,2016-04-16,HOU,CHI,Samantha Johnson,CHI,16,Defender,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False
4,2016,2016-04-16,HOU,CHI,Arin Wright,CHI,3,Defender,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37496,2024,2024-11-03,SD,LOU,Ary Borges,LOU,8,Substitute,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False
37498,2024,2024-11-03,SD,LOU,Ángela Barón,LOU,15,Substitute,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False
37500,2024,2024-11-03,SD,LOU,Milly Clegg,LOU,18,Substitute,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False
37501,2024,2024-11-03,SD,LOU,Parker Goins,LOU,21,Substitute,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False


In [6]:
players = Data.get_nwsl_players()
players

,name,position,birthday,country,hometown,number,height
0,Abby Dahlkemper,Defender,1993-05-13,USA,Lancaster,2.0,"5'6"""
1,Abby Elinsky,Midfielder,1996-01-08,USA,West Chester,20.0,"5'8"""
2,Abby Erceg,Defender,1989-11-20,New Zealand,Whangarei,20.0,"5'9"""
3,Abby Smith,Goalkeeper,1993-10-04,USA,Plano,4.0,"5'8"""
4,Abby Wambach,Attacker,1980-06-02,USA,NaN,NaN,"5'11"""
...,...,...,...,...,...,...,...
988,Rosemonde Kouassi,Midfielder,2001-12-26,Ivory Coast,Tontonou,19.0,"5'5"""
989,Shaelan Murison,Attacker,1998-03-05,USA,NaN,35.0,"5'10"""
990,Tarciane,Defender,2003-05-27,Brazil,Belford Roxo,3.0,"6'0"""
991,Zoe Matthews,Midfielder,2007-05-25,USA,NaN,27.0,"6'2"""


In [7]:
df = stats.set_index("player")[["season", "date", "team", "mins_played"]].join(
    players.set_index("name")[["birthday"]]
)
df = df.dropna()
df = df.reset_index().rename(columns={"index": "player"})
df["date"] = pd.to_datetime(df["date"])
df["birthday"] = pd.to_datetime(df["birthday"])
df

,player,season,date,team,mins_played,birthday
0,Abby Dahlkemper,2016,2016-04-16,WNY,90.0,1993-05-13
1,Abby Dahlkemper,2016,2016-04-23,WNY,90.0,1993-05-13
2,Abby Dahlkemper,2016,2016-04-29,WNY,90.0,1993-05-13
3,Abby Dahlkemper,2016,2016-05-07,WNY,90.0,1993-05-13
4,Abby Dahlkemper,2016,2016-05-14,WNY,90.0,1993-05-13
...,...,...,...,...,...,...
29207,Évelyne Viens,2021,2021-09-25,GFC,9.0,1997-02-06
29208,Évelyne Viens,2021,2021-10-06,GFC,17.0,1997-02-06
29209,Évelyne Viens,2021,2021-10-17,GFC,1.0,1997-02-06
29210,Évelyne Viens,2021,2021-10-28,GFC,6.0,1997-02-06


In [8]:
dob = df.groupby(["player", "season", "birthday"]).agg(
    {"mins_played": "sum"}
).reset_index().rename(
    columns={
        "player": "name",
        "season": "year",
        "mins_played": "minutes",
        "birthday": "dob",
    }
)

dob.to_csv("data/dob_and_minutes.csv", index=False)
dob

,name,year,dob,minutes
0,Abby Dahlkemper,2016,1993-05-13,1800.0
1,Abby Dahlkemper,2017,1993-05-13,2160.0
2,Abby Dahlkemper,2018,1993-05-13,1710.0
3,Abby Dahlkemper,2019,1993-05-13,1260.0
4,Abby Dahlkemper,2021,1993-05-13,720.0
...,...,...,...,...
2079,Zoe Morse,2022,1998-04-01,1889.0
2080,Zoey Goralski,2019,1995-01-22,59.0
2081,Zoey Goralski,2021,1995-01-22,29.0
2082,Ángela Barón,2024,2003-09-18,12.0


In [9]:
df["age"] = df.apply(lambda x: relativedelta(x["date"], x["birthday"]).years, axis=1)
df

,player,season,date,team,mins_played,birthday,age
0,Abby Dahlkemper,2016,2016-04-16,WNY,90.0,1993-05-13,22
1,Abby Dahlkemper,2016,2016-04-23,WNY,90.0,1993-05-13,22
2,Abby Dahlkemper,2016,2016-04-29,WNY,90.0,1993-05-13,22
3,Abby Dahlkemper,2016,2016-05-07,WNY,90.0,1993-05-13,22
4,Abby Dahlkemper,2016,2016-05-14,WNY,90.0,1993-05-13,23
...,...,...,...,...,...,...,...
29207,Évelyne Viens,2021,2021-09-25,GFC,9.0,1997-02-06,24
29208,Évelyne Viens,2021,2021-10-06,GFC,17.0,1997-02-06,24
29209,Évelyne Viens,2021,2021-10-17,GFC,1.0,1997-02-06,24
29210,Évelyne Viens,2021,2021-10-28,GFC,6.0,1997-02-06,24


In [10]:
minutes = (
    df.rename(columns={"season": "year"})
    .groupby(["team", "year", "age"])
    .agg({"mins_played": "sum"})
    .reset_index()
    .pivot(index=["team", "year"], columns="age", values="mins_played")
)

minutes.to_csv("data/nwsl_minutes.csv")
minutes

age           15     16  17    18      19      20      21      22      23  \
team year                                                                   
BAY  2024    NaN    NaN NaN   NaN  1109.0   238.0     2.0    14.0  2098.0   
BOS  2016    NaN    NaN NaN   NaN     NaN     NaN   100.0  1464.0  3273.0   
     2017    NaN    NaN NaN   NaN     NaN     NaN  1559.0  1626.0  4395.0   
CHI  2016    NaN    NaN NaN   NaN     NaN     NaN     NaN  1648.0  6799.0   
     2017    NaN    NaN NaN   NaN     NaN     NaN   134.0   327.0  2772.0   
...          ...    ...  ..   ...     ...     ...     ...     ...     ...   
WAS  2021    NaN    NaN NaN  87.0  1639.0     NaN   657.0  3653.0  1076.0   
     2022    NaN    NaN NaN   NaN   270.0  1234.0     NaN   300.0  6404.0   
     2023  150.0  426.0 NaN   NaN     NaN   624.0  1011.0  1731.0  1965.0   
     2024    NaN   36.0 NaN   NaN     NaN     NaN  2607.0  2333.0  4427.0   
WNY  2016    NaN    NaN NaN   NaN     NaN    45.0  1388.0  4582.0  6445.0   

age            24  ...      33     34  35  36  37  38  39  40     41  42  
team year          ...                                                    
BAY  2024  2802.0  ...   214.0    NaN NaN NaN NaN NaN NaN NaN    NaN NaN  
BOS  2016  4420.0  ...     NaN    NaN NaN NaN NaN NaN NaN NaN    NaN NaN  
     2017  5899.0  ...   804.0    NaN NaN NaN NaN NaN NaN NaN    NaN NaN  
CHI  2016  3092.0  ...     NaN    NaN NaN NaN NaN NaN NaN NaN    NaN NaN  
     2017  6915.0  ...     NaN    NaN NaN NaN NaN NaN NaN NaN    NaN NaN  
...           ...  ...     ...    ...  ..  ..  ..  ..  ..  ..    ...  ..  
WAS  2021  3558.0  ...   653.0    NaN NaN NaN NaN NaN NaN NaN    NaN NaN  
     2022   708.0  ...   195.0  180.0 NaN NaN NaN NaN NaN NaN    NaN NaN  
     2023  4182.0  ...    19.0    NaN NaN NaN NaN NaN NaN NaN  181.0 NaN  
     2024  1881.0  ...  1226.0  455.0 NaN NaN NaN NaN NaN NaN    NaN NaN  
WNY  2016  2216.0  ...     NaN    NaN NaN NaN NaN NaN NaN NaN    NaN NaN  

[86 rows x 28 columns]